## Importing Libraries And Setting Working Directory

In [3]:
import pandas as pd
import os # use this to access your environment variables
import requests # this will be used to call the APIs
import numpy as np

In [5]:
import time

In [4]:
os.chdir("C:/Users/Laven/Documents/Data_Analysis/Capstone_Project")

## Getting API Keys

In [6]:
google_api_key = os.getenv('GOOGLE_PLACES_API_KEY')

In [7]:
yelp_api_key = os.getenv('YELP_API_KEY')


## Querying Database

In [8]:
google_parameters = {
    'query': 'light rail station in Montreal',
    'key': google_api_key,
    'location': '45.5088,-73.5878',
    'radius': 30000,  
}


In [14]:
response = requests.get('https://maps.googleapis.com/maps/api/place/textsearch/json', params=google_parameters)


In [15]:
response_json = response.json()
next_page_token = response_json.get('next_page_token')

In [16]:
next_page_token

'ATplDJZNZvwTiW33hZLKjfzMdSH3hE4ketVgIBMGQjVwy5H0QfiPH1Z5UpNb834Vd19GGJUpQv0UZ0HXrhSMhqD2xUD6_LCeaHuJFT22gCzUg0KOpx4vp0OGz3_8sqoWgYJA0L6z_4hgis2RnwgAG5Btp7Xme3NdzQixIJZ3iUtTAsRKFGLn7eWDhLz9aq22FCcwUvm63lbhGKcCJOhgWk2poLvTjfxsHSBCwD-CcWCZtokKufJ_Fhq8I2lPhFgYjR_R_D0vtba3ZiLhjVAexVYcajYblhpNo_EiG978c6ctmVH_jiL_ATJGGCMCOxaSmVs4tdJL-vdtOBC5C9mvnoQkaGv9RHuJVVzUCUpsclWqqXs0ayl9-FriH7kLn1EIT2Du7DYwv5RqH2im3vnTAHCjJ26376kAs4Dc8maFOEx_dBHp7FKAYMkNB5Uh50WQE5WSCLCa3UUKNFWTANaWu_Xe_y_dHmz3zg'

In [18]:
response = requests.get('https://maps.googleapis.com/maps/api/place/textsearch/json', params=google_parameters)
response_json = response.json()
results = response_json.get('results', [])

# Process the first page of results
data = [{
    'Name': place['name'],
    'Address': place['formatted_address'],
    'Latitude': place['geometry']['location']['lat'],
    'Longitude': place['geometry']['location']['lng']
} for place in results]

# Get the next page token, if available
next_page_token = response_json.get('next_page_token')

while next_page_token:
    time.sleep(2)  # Sleep to give API time to prepare the next set of results
    next_params = {
        'pagetoken': next_page_token,
        'key': google_api_key
    }
    next_response = requests.get('https://maps.googleapis.com/maps/api/place/textsearch/json', params=next_params)
    next_response_json = next_response.json()
    
    # Append new results to the existing data
    new_results = next_response_json.get('results', [])
    data.extend([{
        'Name': place['name'],
        'Address': place['formatted_address'],
        'Latitude': place['geometry']['location']['lat'],
        'Longitude': place['geometry']['location']['lng']
    } for place in new_results])
    
    # Update the next page token
    next_page_token = next_response_json.get('next_page_token')

# Convert all the results to a DataFrame
montreal_light_rail_df = pd.DataFrame(data)
print(montreal_light_rail_df.head(50))  # Adjust number to display as needed

                                      Name  \
0                            Gare Centrale   
1                    Central Train Station   
2                           Place-des-Arts   
3                   Station Île-des-Soeurs   
4              Train Station Gare Centrale   
5                    Montréal West Station   
6               REM ÎLE-DES-SOEURS Station   
7                                   McGill   
8                             De Castelnau   
9                                    Sauvé   
10                                  Dorval   
11                                Ahuntsic   
12                               Angrignon   
13  VIA Rail - Montreal Maintenance Centre   
14         Central Station Complex-Homburg   
15                        VIA RAIL STATION   
16                             Bonaventure   
17                       Dorval (VIA Rail)   
18                   Terminus Centre-Ville   
19                             Montpellier   
20                         VIA Rai

In [19]:
montreal_light_rail_df


,Name,Address,Latitude,Longitude
0,Gare Centrale,"895 Rue De la Gauchetière O, Montréal, QC H3B ...",45.500112,-73.566522
1,Central Train Station,"1001 Pl. Jean-Paul-Riopelle, Montréal, QC H2Z ...",45.499460,-73.564974
2,Place-des-Arts,"Jeanne-Mance St, Montreal, Quebec, Canada",45.507791,-73.568813
3,Station Île-des-Soeurs,"Montreal, QC, Canada",45.470325,-73.538879
4,Train Station Gare Centrale,"935 Rue De la Gauchetière O, Montréal, QC H3B ...",45.499327,-73.566726
5,Montréal West Station,"7499 Ave Harley, Montreal, Quebec H4B 1L7, Canada",45.453446,-73.641674
6,REM ÎLE-DES-SOEURS Station,"900 Bd René Lévesque, Verdun, QC H3E 1H5, Canada",45.470360,-73.538467
7,McGill,"Montreal, QC H3A 2A4, Canada",45.504052,-73.571523
8,De Castelnau,"1 Rue de Castelnau O, Montréal, QC H2R 2X9, Ca...",45.535437,-73.619873
9,Sauvé,"Montreal, Quebec H3L 2B8, Canada",45.549552,-73.654346


## Adding GeoHashes

In [20]:
import geohash

# Function to generate geohash
def compute_geohash(row, precision=5):
    return geohash.encode(row['Latitude'], row['Longitude'], precision=precision)


In [21]:
# Apply the function to each row in the DataFrame
montreal_light_rail_df['Geohash'] = montreal_light_rail_df.apply(compute_geohash, axis=1)


In [22]:
montreal_light_rail_df

,Name,Address,Latitude,Longitude,Geohash
0,Gare Centrale,"895 Rue De la Gauchetière O, Montréal, QC H3B ...",45.500112,-73.566522,f25dv
1,Central Train Station,"1001 Pl. Jean-Paul-Riopelle, Montréal, QC H2Z ...",45.499460,-73.564974,f25dv
2,Place-des-Arts,"Jeanne-Mance St, Montreal, Quebec, Canada",45.507791,-73.568813,f25dv
3,Station Île-des-Soeurs,"Montreal, QC, Canada",45.470325,-73.538879,f25dw
4,Train Station Gare Centrale,"935 Rue De la Gauchetière O, Montréal, QC H3B ...",45.499327,-73.566726,f25dv
5,Montréal West Station,"7499 Ave Harley, Montreal, Quebec H4B 1L7, Canada",45.453446,-73.641674,f25ds
6,REM ÎLE-DES-SOEURS Station,"900 Bd René Lévesque, Verdun, QC H3E 1H5, Canada",45.470360,-73.538467,f25dw
7,McGill,"Montreal, QC H3A 2A4, Canada",45.504052,-73.571523,f25dv
8,De Castelnau,"1 Rue de Castelnau O, Montréal, QC H2R 2X9, Ca...",45.535437,-73.619873,f25eh
9,Sauvé,"Montreal, Quebec H3L 2B8, Canada",45.549552,-73.654346,f25e5


In [23]:
unique_geohashes = montreal_light_rail_df['Geohash'].nunique()
unique_geohashes

11

In [24]:
montreal_light_rail_df.to_csv('montreal_light_rail.csv', index=False)